# Prepare Data

## Install Required Libraries

In [ ]:
!pip install kaggle --upgrade -q
!pip install wandb --upgrade -q

## Download kaggle files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! mkdir ~/.kaggle

In [ ]:
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle competitions download -c walmart-recruiting-store-sales-forecasting

## Unzip Data

In [ ]:
! unzip /content/walmart-recruiting-store-sales-forecasting.zip

## Imports

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import numpy as np
import wandb
from PIL import Image

## Read data frames

In [ ]:
sample_subm_df = pd.read_csv('/content/sampleSubmission.csv')
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/test.csv')
features_df = pd.read_csv('/content/features.csv')
store_df = pd.read_csv('/content/stores.csv')